In [ ]:
Summer Term  
Duke University, The Fuqua School of Business  
**Data Infrastructure project**  
Author:Yiyang Zhang(Under the guidance of professor Ryan at the Fuqua School of Business)

In [ ]:
# Window Functions

* Window functions allow us to aggregate data as we might with a **GROUP BY** clause 
* But, unlike **GROUP BY**, window functions do NOT restrict us to having one row of output per group

In [ ]:
import pymysql
pymysql.install_as_MySQLdb()
%reload_ext sql
%sql mysql://:@mqm-db/
%sql USE airline_ontime;

In [ ]:
%%sql

SELECT *
FROM   ontime
LIMIT  10;

In [ ]:
## Starting Simple...
* Suppose we use an aggregate function with **OVER()** and nothing else...
```mySQL
WITH RDU_F AS
(
SELECT   *
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   Origin, 
           COUNT(*) OVER()
FROM     ontime
LIMIT    10;
```


In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   Origin, 
         COUNT(*) OVER()
FROM     ontime
LIMIT    10;

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   Origin, 
         COUNT(*) OVER()
FROM     RDU_F
LIMIT    10;

* To confirm our suspicions, let's just do a basic COUNT(*)



In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   COUNT(*)
FROM     RDU_F;

In [ ]:
* So, if we use **OVER()** without any arguments, it aggregates over all records in the *result set*


In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   Origin, 
         COUNT(*) OVER()
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
LIMIT    10;

In [ ]:
* Why has our count decreased?  Because all of the cancelled and diverted flights are filted in the WHERE clause **before** the window function is processed.
* In fact, window functions are processed just prior to ORDER BY.  So, WHERE, GROUP BY, and HAVING all come first.
* Speaking of GROUP BY

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   UniqueCarrier,
         COUNT(*),
         COUNT(*) OVER(),
         SUM(Distance) OVER()
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
GROUP BY UniqueCarrier;

In [ ]:
* So, a GROUP BY can play nicely with window functions but the GROUP BY is still being processed first. we simply define a "partition" (i.e. a group of rows) that the window function will aggregate.  To do this we use the **PARTITION BY** clause inside of **OVER()**


In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   UniqueCarrier,
         DepTS,
         COUNT(*) OVER(PARTITION BY UniqueCarrier)
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
ORDER BY DepTS
LIMIT    200;

* Notice that now we can have multiple records for each UniqueCarrier but the COUNT(\*) for each UniqueCarrier still matches our output from the GROUP BY query
* Also, we can now include whatever columns we want in the SELECT clause without needing them to match a column list in the GROUP BY clause!
* Additionally, it is perfectly fine to PARTITION BY multiple columns...

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   UniqueCarrier,
         DepTS,
         COUNT(*) OVER(PARTITION BY UniqueCarrier, DATE(DepTS))
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
ORDER BY DepTS
LIMIT    200;

* **NULL** values are handled in the same manner as they would be by a GROUP BY...
* (Remember that in Jupyter a **NULL** is denoted by "None")

In [ ]:
%%sql

SELECT    a1c,
          COUNT(*)
FROM      sanford.health
GROUP BY  a1c
ORDER BY  COUNT(*) DESC
LIMIT     10;

In [ ]:
%%sql

SELECT    a1c,
          COUNT(*) OVER(PARTITION BY a1c)
FROM      sanford.health
LIMIT     10;

In [ ]:
%%sql

SELECT    a1c,
          COUNT(a1c) OVER(PARTITION BY a1c)
FROM      sanford.health
LIMIT     10;

* Let's see if we're actually comprehending what's going on here
* Use window functions to calculate five average delays...
  * Average departure delay for all airlines for the whole year
  * Average departure delay for an airline on the current day
  * Average departure delay for all airlines on the current day
  * Average departure delay for an airline for the whole year
  * Average departure delay for an airline on the current day of the week

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   UniqueCarrier,
         DepTS,
         DepDelay,
         #Your code goes here!
         AVG(DepDelay) OVER() AS Avg_All,
         AVG(DepDelay) OVER(PARTITION BY UniqueCarrier, DATE(DepTS)) AS Avg_Carrier_Today,
         AVG(DepDelay) OVER(PARTITION BY DATE(DepTS)) AS Avg_Today,
         AVG(DepDelay) OVER(PARTITION BY UniqueCarrier) AS Avg_Carrier,
         AVG(DepDelay) OVER(PARTITION BY UniqueCarrier, DAYOFWEEK(DepTS)) AS Avg_Carrier_DOW
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
ORDER BY DepTS
LIMIT    200;

In [ ]:

* Suppose we care about the ordering of the rows in each partition that we are passing to the window function.  Can our window functions handle this situation???  Of course!  In addition to the **PARTITION BY** subclause, we can also specify an **ORDER BY** subclause within **OVER()**


In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   UniqueCarrier,
         DepTS,
         DepDelay,
         SUM(DepDelay) OVER(ORDER BY DepTS),
         COUNT(DepDelay) OVER(ORDER BY DepTS)
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1 AND
         DATE(DepTS) = DATE('2007-08-11')
ORDER BY DepTS
LIMIT    200;

In [ ]:
* So, our SUM() is actually counting a running total of the day's departure delays, ordered by time
* How/why did it know to do this?  

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   UniqueCarrier,
         DepTS,
         DepDelay,
         SUM(DepDelay) OVER(ORDER BY DepTS),
         COUNT(DepDelay) OVER(ORDER BY DepTS)
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1 AND
         DATE(DepTS) = '2007-08-11'
LIMIT    200;

* Comparing values we can see that the ORDER BY in our query makes no difference--the values are identical!
* Even if we concede that the optimizer may be ordering records by DepTS, it still is not clear why including ORDER BY in our **OVER()** clause should generate a running total (or count)
* Well, it's actually because if we specify **OVER(PARTITION BY X ORDER BY Y)**, we are using the default "framing clause" (in terms of the default, you can think of it as being similar to the default partition when we utilize **OVER()** without a **PARTITION BY**)
* The default framing clause considers ***all rows before, up to and including the current row, where the ordering is specified by the ORDER BY subclause***.  So, this default is essentially the definition of a "running count" or "running total" (and is therefore the reason why those were calculated in the previous query).
* Why call it a "framing clause?"  Because we can visualize the window function as performing aggregate calculations on a moving "window frame."  The framing clause is what specifies the size, movement of the window frame.
* In fact the previous query can be rewritten as...


In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
)
SELECT   UniqueCarrier,
         DepTS,
         DepDelay,

         SUM(DepDelay) OVER(ORDER BY DepTS ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW),
         COUNT(DepDelay) OVER(ORDER BY DepTS ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1 AND
         DATE(DepTS) = '2007-08-11'
ORDER BY DepTS
LIMIT    200;

In [ ]:
*What are all of our options in specifying the framing clause?

```mySQL
SELECT   AGG_FN() OVER(PARTITION BY ...
                        ORDER BY ...
                        ROWS/RANGE BETWEEN UNBOUNDED PRECEDING/
                                           X PRECEDING/
                                           CURRENT ROW/
                                           X FOLLOWING
                                       AND X PRECEDING/
                                           CURRENT ROW/
                                           X FOLLOWING/
                                           UNBOUNDED FOLLOWING
                        )
FROM     Tablename;
```
* You are able to choose any of the four options after the "BETWEEN" as well as any of the four options after the "AND", so long as you are working from "earlier" to "later" records.  <mark>***Meaning, you can specify ROWS BETWEEN 4 PRECEDING AND 2 FOLLOWING but you CANNOT specify ROWS BETWEEN 2 FOLLOWING AND 4 PRECEDING.***</mark>
* ***NOTE:*** there is a very subtle difference in the choice of ***ROWS*** versus ***RANGE***
    * ***ROWS*** WILL arbitrarily break ties in your ***ORDER BY*** subclause 
    * ***RANGE*** WILL NOT arbitrarily break ties in your ***ORDER BY*** subclause.  Instead, it will consider all "tied records" together.  ALSO, ***RANGE*** only works if the fields in your ***ORDER BY*** subclause are all numeric.  (And this limitation will typically force you to choose ***ROWS*** over ***RANGE***.)
* So, how would we calculate a 5-day moving average of average carrier delays (for each carrier)?

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
),
AVG_DEL AS
(
SELECT   UniqueCarrier,
         DATE(DepTS) AS Date_Col,
         AVG(DepDelay) AS AD
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
GROUP BY UniqueCarrier,
         DATE(DepTS)
)
SELECT   UniqueCarrier,
         Date_Col,
         AD,
         AVG(AD) OVER(PARTITION BY UniqueCarrier 
                      ORDER BY Date_Col ROWS BETWEEN 4 PRECEDING 
                                          AND CURRENT ROW
                     )
FROM     AVG_DEL
ORDER BY UniqueCarrier,
         Date_Col
LIMIT    200;

* And just to clarify the difference between ***ROWS*** and ***RANGE***, let's run a few queries using our trusty "Reviews" table...

In [ ]:
%sql USE lecture2;

In [ ]:
%%sql

SELECT     *
FROM       Reviews
ORDER BY   RateDate;

In [ ]:
%%sql
# Notice that ROWS will arbitrarily break the tie between records with the same RateDate (our lone column in
# the ORDER BY subclause).  We can remedy this (in next cell) by also including ReviewID in our ORDER BY subclause.
SELECT      *,
            AVG(Rating) OVER(ORDER BY RateDate ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING)
FROM        Reviews
ORDER BY    RateDate;

In [ ]:
%%sql

SELECT     *
FROM       Reviews
ORDER BY   RateDate,
           ReviewID;

In [ ]:
%%sql

SELECT      *,
            AVG(Rating) OVER(ORDER BY RateDate, ReviewID ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING)
FROM        Reviews
ORDER BY    RateDate,
            ReviewID;

In [ ]:
%%sql

SELECT     *
FROM       Reviews
ORDER BY   Profile,
           RateDate,
           ReviewID;

In [ ]:
%%sql

SELECT      *,
            AVG(Rating) OVER(PARTITION BY Profile ORDER BY RateDate, ReviewID ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING)
FROM        Reviews
ORDER BY    RateDate,
            ReviewID;

In [ ]:
%%sql

SELECT      *,
            AVG(Rating) OVER(PARTITION BY Profile ORDER BY RateDate, ReviewID ROWS BETWEEN 2 FOLLOWING AND 3 FOLLOWING)
FROM        Reviews
ORDER BY    RateDate,
            ReviewID;

In [ ]:
* it's time for the creme de la creme of SQL queries in this course.  Suppose that RDU penalizes airline carriers for departure delays (whether or not they are indeed that carrier's fault) as follows:
  * Each day the airport calculates the average departure delay for each carrier.
  * Next, for each day, carriers are ranked from highest to lowest average departure delay.  To be included in the ranking a carrier must have at least 5 departures on a given day.  If a carrier has fewer than 5 departures on a given day then it is automatically ranked tied for last on that day.
  * Finally, a carrier experiences a "penalty day" on any day when it was ranked in the Top 2 on at least 2 of the previous 3 days.  (We can imagine that the flight deck might give runway priority to the other airlines on a penalty day.)
* So, given all of this, which carrier has the most penalty days during 2007?

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
),
AVG_DEL AS
(
SELECT   UniqueCarrier,
         DATE(DepTS) AS Date_Col,
         AVG(DepDelay) AS AD,
         COUNT(*) AS Num_Flights
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
GROUP BY UniqueCarrier,
         DATE(DepTS)
)
SELECT   *,
         RANK() OVER(PARTITION BY Date_Col
                     ORDER BY AD DESC) AS Delay_Rank
FROM     AVG_DEL
ORDER BY Date_Col,
         Delay_Rank
LIMIT    200;

* Okay, so in the above query we now see how to **RANK()** carriers.  However, there is a problem--we're still ranking carriers with fewer than 5 flights in a day...

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
),
AVG_DEL AS
(
SELECT   UniqueCarrier,
         DATE(DepTS) AS Date_Col,
         AVG(DepDelay) AS AD,
         COUNT(*) AS Num_Flights
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
GROUP BY UniqueCarrier,
         DATE(DepTS)
)
SELECT   *,
         RANK() OVER(PARTITION BY Date_Col
                     ORDER BY AD DESC) AS Delay_Rank
FROM     AVG_DEL
WHERE    Num_Flights >= 5
ORDER BY Date_Col,
         Delay_Rank
LIMIT    2000;

* Problem with the above query is that if a carrier has fewer than 5 flights in a day, it now isn't given a ranking at all...

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
),
AVG_DEL AS
(
SELECT   UniqueCarrier,
         DATE(DepTS) AS Date_Col,
         AVG(DepDelay) AS AD,
         COUNT(*) AS Num_Flights
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
GROUP BY UniqueCarrier,
         DATE(DepTS)
)
SELECT   *,
         CASE
         WHEN Num_Flights >= 5 
         THEN RANK() OVER(PARTITION BY Date_Col
                          ORDER BY AD DESC)
         ELSE COUNT(*) OVER(PARTITION BY Date_Col)
         END  AS Delay_Rank
FROM     AVG_DEL
ORDER BY Date_Col,
         Delay_Rank
LIMIT    100;

<mark>***The below query finally does what we want it do!  Specifically, we first rank only the carriers that have at least 5 flights on a day.  Then we use a LEFT JOIN with COALESCE() to fill in the rankings of all carriers with fewer than 5 flights on each day.  Notice that we do not need an ORDER BY subclause with the MAX() window function because we already took care of the rankings in the previous CTE.***</mark>

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
),
AVG_DEL AS
(
SELECT   UniqueCarrier,
         DATE(DepTS) AS Date_Col,
         AVG(DepDelay) AS AD,
         COUNT(*) AS Num_Flights
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
GROUP BY UniqueCarrier,
         DATE(DepTS)
),
DEL_RANK AS
(
SELECT   *,
         RANK() OVER(PARTITION BY Date_Col
                     ORDER BY AD DESC) AS Delay_Rank
FROM     AVG_DEL
WHERE    Num_Flights >= 5
)
SELECT          *,
                COALESCE(Delay_Rank, MAX(Delay_Rank) OVER(PARTITION BY AVG_DEL.Date_Col)) AS New_Delay_Rank
FROM            AVG_DEL
LEFT JOIN       DEL_RANK
ON              AVG_DEL.UniqueCarrier = DEL_RANK.UniqueCarrier AND
                AVG_DEL.Date_Col = DEL_RANK.Date_Col
ORDER BY AVG_DEL.Date_Col,
         New_Delay_Rank
LIMIT    200;

<mark>***Next, the below query creates a new column called "Issue_Flag" which we assign a value of 1 any time a carrier is ranked 1st or 2nd on each day.***</mark>

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
),
AVG_DEL AS
(
SELECT   UniqueCarrier,
         DATE(DepTS) AS Date_Col,
         AVG(DepDelay) AS AD,
         COUNT(*) AS Num_Flights
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
GROUP BY UniqueCarrier,
         DATE(DepTS)
),
DEL_RANK AS
(
SELECT   *,
         RANK() OVER(PARTITION BY Date_Col
                     ORDER BY AD DESC) AS Delay_Rank
FROM     AVG_DEL
WHERE    Num_Flights >= 5
),
COR_DEL_RANK AS
(
SELECT          AVG_DEL.UniqueCarrier,
                AVG_DEL.Date_Col,
                COALESCE(Delay_Rank, MAX(Delay_Rank) OVER(PARTITION BY AVG_DEL.Date_Col)) AS New_Delay_Rank
FROM            AVG_DEL
LEFT JOIN       DEL_RANK
ON              AVG_DEL.UniqueCarrier = DEL_RANK.UniqueCarrier AND
                AVG_DEL.Date_Col = DEL_RANK.Date_Col
ORDER BY AVG_DEL.Date_Col,
         New_Delay_Rank
)
SELECT          COR_DEL_RANK.Date_Col,
                COR_DEL_RANK.UniqueCarrier,
                COR_DEL_RANK.New_Delay_Rank,
                CASE   WHEN COR_DEL_RANK.New_Delay_Rank <= 2 THEN 1
                ELSE   0
                END AS Issue_Flag
FROM     COR_DEL_RANK
ORDER BY COR_DEL_RANK.Date_Col,
         COR_DEL_RANK.New_Delay_Rank
LIMIT    2000;

<mark>***Next, in the below query we create a "running sum" of the number of "issue days" that occur in the previous 3 days.  Each day where this value is >= 2 will denote an actual penalty.***</mark>

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
),
AVG_DEL AS
(
SELECT   UniqueCarrier,
         DATE(DepTS) AS Date_Col,
         AVG(DepDelay) AS AD,
         COUNT(*) AS Num_Flights
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
GROUP BY UniqueCarrier,
         DATE(DepTS)
),
DEL_RANK AS
(
SELECT   *,
         RANK() OVER(PARTITION BY Date_Col
                     ORDER BY AD DESC) AS Delay_Rank
FROM     AVG_DEL
WHERE    Num_Flights >= 5
),
COR_DEL_RANK AS
(
SELECT          AVG_DEL.UniqueCarrier,
                AVG_DEL.Date_Col,
                COALESCE(Delay_Rank, MAX(Delay_Rank) OVER(PARTITION BY AVG_DEL.Date_Col)) AS New_Delay_Rank
FROM            AVG_DEL
LEFT JOIN       DEL_RANK
ON              AVG_DEL.UniqueCarrier = DEL_RANK.UniqueCarrier AND
                AVG_DEL.Date_Col = DEL_RANK.Date_Col
ORDER BY AVG_DEL.Date_Col,
         New_Delay_Rank
), DEL_FLAG AS
(
SELECT          COR_DEL_RANK.Date_Col,
                COR_DEL_RANK.UniqueCarrier,
                COR_DEL_RANK.New_Delay_Rank,
                CASE   WHEN COR_DEL_RANK.New_Delay_Rank <= 2 THEN 1
                ELSE   0
                END AS Issue_Flag
FROM     COR_DEL_RANK
)
SELECT   UniqueCarrier,
         Date_Col,
         New_Delay_Rank,
         Issue_Flag,
         SUM(Issue_Flag) OVER(PARTITION BY UniqueCarrier
                              ORDER BY Date_Col
                              ROWS BETWEEN 3 PRECEDING AND
                              1 PRECEDING) AS RUN_COUNT_PROB
FROM     DEL_FLAG
ORDER BY UniqueCarrier,
         Date_Col
LIMIT    2000;

<mark>***Finally, we tabulate the penalty days below.***</mark>

In [ ]:
%%sql

WITH RDU_F AS
(
SELECT   *,
         CASE
         WHEN DepTime < 60 THEN
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',DepTime), '%Y %c %e %i')
         ELSE
         STR_TO_DATE(CONCAT(Year,' ',Month,' ',DayOfMonth,' ',SUBSTR(DepTime,1,LENGTH(DepTime)-2),' ',SUBSTR(DepTime,-2)), '%Y %c %e %k %i')
         END as DepTS
FROM     ontime
WHERE    TRIM(Origin) = 'RDU'
),
AVG_DEL AS
(
SELECT   UniqueCarrier,
         DATE(DepTS) AS Date_Col,
         AVG(DepDelay) AS AD,
         COUNT(*) AS Num_Flights
FROM     RDU_F
WHERE    Cancelled != 1 AND
         Diverted != 1
GROUP BY UniqueCarrier,
         DATE(DepTS)
),
DEL_RANK AS
(
SELECT   *,
         RANK() OVER(PARTITION BY Date_Col
                     ORDER BY AD DESC) AS Delay_Rank
FROM     AVG_DEL
WHERE    Num_Flights >= 5
),
COR_DEL_RANK AS
(
SELECT          AVG_DEL.UniqueCarrier,
                AVG_DEL.Date_Col,
                COALESCE(Delay_Rank, MAX(Delay_Rank) OVER(PARTITION BY AVG_DEL.Date_Col)) AS New_Delay_Rank
FROM            AVG_DEL
LEFT JOIN       DEL_RANK
ON              AVG_DEL.UniqueCarrier = DEL_RANK.UniqueCarrier AND
                AVG_DEL.Date_Col = DEL_RANK.Date_Col
ORDER BY AVG_DEL.Date_Col,
         New_Delay_Rank
), DEL_FLAG AS
(
SELECT          COR_DEL_RANK.Date_Col,
                COR_DEL_RANK.UniqueCarrier,
                COR_DEL_RANK.New_Delay_Rank,
                CASE   WHEN COR_DEL_RANK.New_Delay_Rank <= 2 THEN 1
                ELSE   0
                END AS Issue_Flag
FROM     COR_DEL_RANK
),
LAST_CTE AS
(
SELECT   UniqueCarrier,
         Date_Col,
         New_Delay_Rank,
         Issue_Flag,
         SUM(Issue_Flag) OVER(PARTITION BY UniqueCarrier
                              ORDER BY Date_Col
                              ROWS BETWEEN 3 PRECEDING AND
                              1 PRECEDING) AS RUN_COUNT_PROB
FROM     DEL_FLAG
)
SELECT   UniqueCarrier,
         COUNT(*)
FROM     LAST_CTE
WHERE    RUN_COUNT_PROB >= 2
GROUP BY UniqueCarrier
ORDER BY COUNT(*) DESC;